In [ ]:
# @title 1. Initialize & Connect to Drive (Step 1)🚀
import os
import sys
import logging

# --- 🎨 Color & Emoji Class ---
class Style:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    RESET = '\033[0m'
    BOLD = '\033[1m'

print(f"{Style.HEADER}{Style.BOLD}🔄 Initializing Environment...{Style.RESET}")

# --- Install Dependencies ---
try:
    import yt_dlp
    print(f"{Style.GREEN}✅ yt-dlp is already installed.{Style.RESET}")
except ImportError:
    print(f"{Style.CYAN}⬇️  Installing yt-dlp...{Style.RESET}")
    !pip install yt_dlp -q
    import yt_dlp
    print(f"{Style.GREEN}✅ Installation complete.{Style.RESET}")

# --- Mount Google Drive ---
from google.colab import drive

def mount_drive():
    drive_path = '/content/drive'
    if os.path.exists(drive_path):
        print(f"{Style.GREEN}✅ Google Drive is already mounted.{Style.RESET}")
        return drive_path

    try:
        print(f"{Style.YELLOW}📂 Please authorize Google Drive access...{Style.RESET}")
        drive.mount(drive_path)
        print(f"{Style.GREEN}✅ Drive mounted successfully!{Style.RESET}")
        return drive_path
    except Exception as e:
        print(f"{Style.RED}❌ Failed to mount Drive.{Style.RESET}")
        sys.exit("Critical Error: Cannot proceed without Drive.")

# Execute Mount
DRIVE_ROOT = mount_drive()
DOWNLOAD_FOLDER = os.path.join(DRIVE_ROOT, "My Drive", "YouTube_Downloads")

# Create folder if missing
if not os.path.exists(DOWNLOAD_FOLDER):
    try:
        os.makedirs(DOWNLOAD_FOLDER)
        print(f"{Style.CYAN}📁 Created folder: {DOWNLOAD_FOLDER}{Style.RESET}")
    except OSError as e:
        print(f"{Style.RED}❌ Error creating folder.{Style.RESET}")

In [ ]:
# @title 2. Download Video (Step 2 Select Quality) 🎬
# @markdown Select your desired quality, enter the URL, and press Play.

target_url = "https://www.youtube.com/watch?v=EXAMPLE" # @param {type:"string"}
video_quality = "1080p" # @param ["360p", "480p", "720p", "1080p", "1440p", "2160p (4K)", "4320p (8K)"]

import yt_dlp
import os
import sys
Audio = False
# --- 🎨 Style Definitions ---
class Style:
    CYAN = '\033[96m'   
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    RESET = '\033[0m'
    BOLD = '\033[1m'

# --- 📊 Custom Progress Hook ---
def progress_hook(d):
    global Audio
    if d['status'] == 'downloading':
        total = d.get('total_bytes') or d.get('total_bytes_estimate', 0)
        downloaded = d.get('downloaded_bytes', 0)
        
        if total > 0:
            percent = downloaded / total * 100
        else:
            percent = 0
            
        bar_length = 25
        filled_length = int(bar_length * percent // 100)
        bar = '▓' * filled_length + '░' * (bar_length - filled_length)
        
        speed = d.get('_speed_str', 'N/A')
        eta = d.get('_eta_str', 'N/A')

        sys.stdout.write(
            f"\r{Style.CYAN}🚀 {Style.BOLD}{percent:.1f}%{Style.RESET} "
            f"|{Style.GREEN}{bar}{Style.RESET}| "
            f"[{Style.YELLOW}{speed}{Style.RESET}] "
            f"[{Style.BLUE}ETA: {eta}{Style.RESET}]"
        )
        sys.stdout.flush()

    elif d['status'] == 'finished':
        if Audio == False :
            sys.stdout.write(f"\n{Style.GREEN}✨ Video Downloaded ! Downloading Audio...{Style.RESET}\n")
            Audio = True
        elif Audio == True :
            sys.stdout.write(f"\n{Style.GREEN}✨ Download Complete! Merging Audio & Video - pls wait ...{Style.RESET}\n")


# --- 🧠 Smart Resolution Logic ---
def get_best_available_format(url, target_q_str):
    """
    Analyzes video metadata to see if target quality exists.
    Returns the correct format string and a status message.
    """
    print(f"{Style.YELLOW}📡 Analyzing video metadata...{Style.RESET}")
    
    # 1. Extract target height (e.g. "1080p" -> 1080)
    target_height = int(target_q_str.replace("p", ""))
    
    with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
        try:
            info = ydl.extract_info(url, download=False)
        except Exception as e:
            return None, None, f"Error fetching metadata: {str(e)}"

        video_title = info.get('title', 'Unknown')
        
        # 2. Scan available formats
        available_heights = set()
        for f in info.get('formats', []):
            if f.get('height'):
                available_heights.add(f['height'])
        
        max_height = max(available_heights) if available_heights else 0

        # 3. Decision Logic
        if target_height in available_heights:
            # Target exists!
            print(f"{Style.GREEN}✅ {target_q_str} is available.{Style.RESET}")
            fmt = f'bestvideo[height={target_height}]+bestaudio/best[height={target_height}]'
            return fmt, video_title, "MATCH"
        else:
            # Target missing!
            print(f"{Style.MAGENTA}⚠️  {target_q_str} NOT available.{Style.RESET}")
            print(f"{Style.MAGENTA}🔄 Falling back to highest available: {Style.BOLD}{max_height}p{Style.RESET}")
            # Just get the absolute best available since target failed
            fmt = 'bestvideo+bestaudio/best'
            return fmt, video_title, "FALLBACK"

# --- 🚀 Main Execution ---
def download_smart(url, output_path, quality_str):
    if not url or "youtube.com" not in url and "youtu.be" not in url:
        print(f"{Style.RED}❌ Invalid URL provided.{Style.RESET}")
        return

    print(f"\n{Style.CYAN}🔍 Processing: {Style.BOLD}{url}{Style.RESET}")

    # 1. Check Availability First
    format_str, title, status = get_best_available_format(url, quality_str)

    if not format_str:
        print(f"{Style.RED}❌ Failed to analyze video.{Style.RESET}")
        return

    print(f"{Style.CYAN}🎞️  Title: {Style.BOLD}{title}{Style.RESET}")

    # 2. Configure Downloader with the Chosen Format
    ydl_opts = {
        'format': format_str,
        'merge_output_format': 'mp4',
        'outtmpl': f'{output_path}/%(title)s [%(id)s].%(ext)s',
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
        'progress_hooks': [progress_hook]
    }

    # 3. Download
    try:
        print(f"{Style.YELLOW}⬇️  Starting download...{Style.RESET}")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
            
            print(f"\n{Style.GREEN}{Style.BOLD}✅ ALL DONE!{Style.RESET}")
            print(f"{Style.GREEN}💾 Saved to: {Style.RESET}{output_path}")

    except yt_dlp.utils.DownloadError as e:
        print(f"\n{Style.RED}❌ Download Failed.{Style.RESET}")
    except Exception as e:
        print(f"\n{Style.RED}❌ Unexpected Error: {e}{Style.RESET}")

# --- Run ---
if 'DOWNLOAD_FOLDER' not in globals():
    # Fallback
    DOWNLOAD_FOLDER = "/content/drive/My Drive/YouTube_Downloads"
    if not os.path.exists("/content/drive"):
         print(f"{Style.RED}⚠️ Drive not connected! Run Cell 1.{Style.RESET}")
    else:
         download_smart(target_url, DOWNLOAD_FOLDER, desired_quality)
else:
    download_smart(target_url, DOWNLOAD_FOLDER, desired_quality)